# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# cities data cleaning
for x in range(2, 6):
    cities.iloc[:, x] = cities.iloc[:, x].str.replace('\s*\[.*\]','')
    cities.iloc[:, x] = cities.iloc[:, x].str.replace(r'([a-z])([A-Z])', r'\1 \2')
    cities.iloc[:, x] = cities.iloc[:, x].replace('—', np.NaN)
    cities.iloc[:, x] = cities.iloc[:, x].replace('', np.NaN)
cities['Population (2016 est.)[8]'] = cities['Population (2016 est.)[8]'].astype(int)
    
# cities dataset for nhl
cities_nhl = cities.iloc[:, [1,5]]
cities_nhl = cities_nhl.dropna(subset=['NHL'])
    
# nhl_df data cleaning
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df = nhl_df[['team', 'W', 'L']]
nhl_df = nhl_df.drop([0, 9, 18, 26]).reset_index(drop = True)
nhl_df['team'] = nhl_df['team'].str.replace('\*', '')
nhl_df['team'] = nhl_df['team'].str.replace('^[A-Z]\S*\s', '')
nhl_df['team'] = nhl_df['team'].str.replace('^(Bay|Jersey|York|Louis|Jose|Angeles)\s', '')
nhl_df = nhl_df.set_index('team')
nhl_df[['W', 'L']] = nhl_df[['W', 'L']].astype(int)
nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W'] +nhl_df['L'])
nhl_df = nhl_df.drop(['W', 'L'], axis = 1)
nhl_df = nhl_df.append(
    pd.DataFrame(
        [
            [(nhl_df.at['Rangers', 'W/L%'] + nhl_df.at['Islanders', 'W/L%'] + nhl_df.at['Devils', 'W/L%'])/3],
            [(nhl_df.at['Kings', 'W/L%'] + nhl_df.at['Ducks', 'W/L%'])/2]
        ],
        columns = ['W/L%'],
        index = ['Rangers Islanders Devils', 'Kings Ducks']
    )
)
nhl_df = nhl_df.drop(['Rangers', 'Islanders', 'Devils', 'Kings', 'Ducks']).reset_index().rename(columns = {'index' : 'NHL'})

def nhl_correlation(): 
    result = pd.merge(cities_nhl, nhl_df, how = 'inner', on = 'NHL')
    result = result.drop(['NHL'], axis = 1)
    global nhl_df_final
    nhl_df_final = result
    
    population_by_region = result['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = result['W/L%'] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    #return population_by_region
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nhl_correlation()

0.012486162921209912

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
nba_df=pd.read_csv("assets/nba.csv")

# cities dataset for nba
cities_nba = cities.iloc[:, [1,4]]
cities_nba = cities_nba.dropna(subset=['NBA']).reset_index(drop = True)
    
# nba_df data cleaning
nba_df = nba_df[nba_df['year'] == 2018].reset_index()
nba_df = nba_df[['team', 'W/L%']]
nba_df['team'] = nba_df['team'].str.replace('\**\s\(\d+\)$', '')
nba_df['team'] = nba_df['team'].str.replace(r'(.*)(\s)(\w+)$', r'\3')
nba_df['team'] = nba_df['team'].str.replace(r'(.*)(\s)(\w+)$', r'\3')
nba_df.iat[17, 0] = 'Trail Blazers'
nba_df['W/L%'] = pd.to_numeric(nba_df['W/L%'])
nba_df = nba_df.set_index('team')
nba_df = nba_df.append(
    pd.DataFrame(
        [
            [(nba_df.at['Knicks', 'W/L%'] + nba_df.at['Nets', 'W/L%'])/2],
            [(nba_df.at['Lakers', 'W/L%'] + nba_df.at['Clippers', 'W/L%'])/2]
        ],
        columns = ['W/L%'],
        index = ['Knicks Nets', 'Lakers Clippers']
    )
)
nba_df = nba_df.drop(['Knicks', 'Nets', 'Lakers', 'Clippers']).reset_index().rename(columns = {'index' : 'NBA'})

def nba_correlation():
    result = pd.merge(cities_nba, nba_df, how = 'inner', on = 'NBA')
    result = result.drop(['NBA'], axis = 1)
    global nba_df_final
    nba_df_final = result
    
    population_by_region = result['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = result['W/L%'] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17636350642182938

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
mlb_df=pd.read_csv("assets/mlb.csv")

#cities dataset for mlb
cities_mlb = cities.iloc[:, [1,3]]
cities_mlb = cities_mlb.dropna(subset=['MLB']).reset_index(drop = True)
    
# mlb_df data cleaning
mlb_df = mlb_df[mlb_df['year'] == 2018].reset_index(drop = True)
mlb_df = mlb_df[['team', 'W-L%']].rename(columns = {'W-L%':'W/L%'})
mlb_df['team'] = mlb_df['team'].str.replace('^\S+\s', '')
mlb_df['team'] = mlb_df['team'].str.replace('^(Bay|Jersey|York|Louis|Jose|Angeles|City|Diego|Francisco)\s', '')
mlb_df['W/L%'] = pd.to_numeric(mlb_df['W/L%'])
mlb_df = mlb_df.set_index('team')
mlb_df = mlb_df.append(
    pd.DataFrame(
        [
            [(mlb_df.at['Yankees', 'W/L%'] + mlb_df.at['Mets', 'W/L%'])/2],
            [(mlb_df.at['Dodgers', 'W/L%'] + mlb_df.at['Angels', 'W/L%'])/2],
            [(mlb_df.at['Giants', 'W/L%'] + mlb_df.at['Athletics', 'W/L%'])/2],
            [(mlb_df.at['Cubs', 'W/L%'] + mlb_df.at['White Sox', 'W/L%'])/2]
        ],
        columns = ['W/L%'],
        index = ['Yankees Mets', 'Dodgers Angels', 'Giants Athletics', 'Cubs White Sox']
    )
)
mlb_df = mlb_df.drop(['Yankees', 'Mets', 'Dodgers', 'Angels', 'Giants', 'Athletics', 'Cubs', 'White Sox']).reset_index().rename(columns = {'index' : 'MLB'})

def mlb_correlation(): 
    result = pd.merge(cities_mlb, mlb_df, how = 'inner', on = 'MLB')
    result = result.drop(['MLB'], axis = 1)
    global mlb_df_final
    mlb_df_final = result
    
    population_by_region = result['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = result['W/L%'] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
nfl_df=pd.read_csv("assets/nfl.csv")

# cities dataset for nfl
cities_nfl = cities.iloc[:, 1:3]
cities_nfl = cities_nfl.dropna(subset=['NFL']).reset_index(drop = True)
    
# nfl_df data cleaning
nfl_df = nfl_df[nfl_df['year'] == 2018].reset_index(drop = True)

nfl_df = nfl_df[nfl_df["team"].str.contains("AFC") == False]
nfl_df = nfl_df[nfl_df["team"].str.contains("NFC") == False]
nfl_df = nfl_df[['team', 'W-L%']].rename(columns = {'W-L%':'W/L%'})
nfl_df['team'] = nfl_df['team'].str.replace('^\S+\s', '')
nfl_df['team'] = nfl_df['team'].str.replace('\W*$', '')
nfl_df['team'] = nfl_df['team'].str.replace('^\S+\s', '')
nfl_df['W/L%'] = pd.to_numeric(nfl_df['W/L%'])
nfl_df = nfl_df.set_index('team')
nfl_df = nfl_df.append(
    pd.DataFrame(
        [
            [(nfl_df.at['Giants', 'W/L%'] + nfl_df.at['Jets', 'W/L%'])/2],
            [(nfl_df.at['Rams', 'W/L%'] + nfl_df.at['Chargers', 'W/L%'])/2],
            [(nfl_df.at['49ers', 'W/L%'] + nfl_df.at['Raiders', 'W/L%'])/2]
        ],
        columns = ['W/L%'],
        index = ['Giants Jets', 'Rams Chargers', '49ers Raiders']
    )
)
nfl_df = nfl_df.drop(['Giants', 'Jets', 'Rams', 'Chargers', '49ers', 'Raiders']).reset_index().rename(columns = {'index' : 'NFL'})

def nfl_correlation(): 
    result = pd.merge(cities_nfl, nfl_df, how = 'inner', on = 'NFL')
    result = result.drop(['NFL'], axis = 1)
    global nfl_df_final
    nfl_df_final = result
    
    population_by_region = result['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
    win_loss_by_region = result['W/L%'] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004282141436393033

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
# Combine all W/L% tables
nhl_df_final = nhl_df_final.rename(columns = {'W/L%' : 'NHL'})
nba_df_final = nba_df_final.rename(columns = {'W/L%' : 'NBA'})
mlb_df_final = mlb_df_final.rename(columns = {'W/L%' : 'MLB'})
nfl_df_final = nfl_df_final.rename(columns = {'W/L%' : 'NFL'})
performance = pd.merge(nhl_df_final, nba_df_final, how = 'outer', on = 'Population (2016 est.)[8]')
performance = pd.merge(performance, mlb_df_final, how = 'outer', on = 'Population (2016 est.)[8]')
performance = pd.merge(performance, nfl_df_final, how = 'outer', on = 'Population (2016 est.)[8]')


def sports_team_performance():
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    for i in sports:
        for j in sports:
            if (i != j) :
                p_values.at[i, j] = stats.ttest_rel(performance[i], performance[j], nan_policy = 'omit')[1]
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
